In [1]:
import json 
import os
import logging 
import concurrent 
import asyncio 
import re 
import requests 
import functools 

from pathlib import Path 
from itertools import chain 
from collections import namedtuple
from pprint import PrettyPrinter

from subgrounds import Subgrounds
from subgrounds.pagination import ShallowStrategy
from web3 import Web3
from concurrent.futures import ThreadPoolExecutor
from pycoingecko import CoinGeckoAPI

from prefect import flow, task
from prefect.client import get_client
from prefect.tasks import task_input_hash
from prefect.filesystems import LocalFileSystem 
from prefect.orion.api.task_runs import read_task_run

import pandas as pd 
import numpy as np 
import altair as alt 
import missingno as miss

from flywheel_util.constants import (
    colors_24,
    colors_28, 
    addresses, 
    url_infura, 
    url_snapshot, 
    url_subgraphs, 
    snapshot_api_max_records_per_request, 
    snapshot_api_max_skip,
)
from flywheel_util.tasks.general import df_to_sql
from flywheel_util.utils.util import (
    ddf, 
    first_row, 
    camel_to_snake, 
    cg_get_market_history, 
    df_cols_camel_to_snake, 
    df_cols_change_prefix, 
    df_sort_cols, 
    remove_prefix, 
    graphql_execute, 
    remove_prefix, 
    remove_prefixes, 
    recursive_index_merge, 
    zip_dfs, 
    query_attrs, 
    compare_sets, 
    compare_cols, 
)
from flywheel_util.utils.w3 import (
    get_verified_abi, 
    erc20_read_contract, 
    verified_contract
)

from sqlalchemy import create_engine
from sqlalchemy import text
engine = create_engine("sqlite+pysqlite:///votium_bribes.db", echo=False, future=True)

# logging.basicConfig(level=logging.INFO)

import logging
logging.basicConfig(level=logging.DEBUG)

pp = PrettyPrinter().pprint

alt.data_transformers.disable_max_rows()

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/eth_utils/toolz.py:2: DeprecationWarning: The toolz.compatibility module is no longer needed in Python 3 and has been deprecated. Please import these utilities directly from the standard library. This module will be removed in a future release.
  from cytoolz import (
/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/web3/_utils/normalizers.py:231: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(eth_abi.__version__) < LooseVersion("2"):


DataTransformerRegistry.enable('default')

In [2]:
w3 = Web3(Web3.HTTPProvider(url_infura))
cg = CoinGeckoAPI()

In [3]:
sg = Subgrounds()
sg_curve_pools = sg.load_subgraph(url_subgraphs.convex.curve_pools) 
sg_curve_vol = sg.load_subgraph(url_subgraphs.convex.curve_vol_mainnet)
sg_votium = sg.load_subgraph(url_subgraphs.votium.bribes) 

## Curve Liquidity for FraxBP + FraxBP Metapools

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from flywheel_util.flows.fraxbp import flow_fraxbp_metapool_data
df_pools, df_coins, df_pool_coin, df_pool_snaps, df_mpool_snaps, df_reserves = flow_fraxbp_metapool_data()

13:19:30.083 | INFO    | prefect.engine - Created flow run 'black-capybara' for flow 'flow-fraxbp-metapool-data'

13:19:30.150 | INFO    | Flow run 'black-capybara' - Created task run 'query_curve_mpools_with_gauge-272fe162-0' for task 'query_curve_mpools_with_gauge'

13:19:30.151 | INFO    | Flow run 'black-capybara' - Submitted task run 'query_curve_mpools_with_gauge-272fe162-0' for execution.

13:19:30.159 | INFO    | Flow run 'black-capybara' - Created task run 'query_curve_pool_vol_snapshots-d13ece1a-0' for task 'query_curve_pool_vol_snapshots'

13:19:30.159 | INFO    | Flow run 'black-capybara' - Submitted task run 'query_curve_pool_vol_snapshots-d13ece1a-0' for execution.

...

13:19:30.710 | INFO    | Flow run 'black-capybara' - Submitted task run 'df_to_sql-e5581f58-2' for execution.

13:19:30.715 | INFO    | Task run 'df_to_sql-e5581f58-3' - Finished in state Completed()

13:19:30.738 | INFO    | Task run 'df_to_sql-e5581f58-2' - Finished in state Completed()

13:19:30.761 | INFO    | Flow run 'black-capybara' - Finished in state Completed()

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x2ad39b760>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x29dc61d80>


In [6]:
d = (
    df_mpool_snaps.loc[~df_mpool_snaps.total_apr.isna()]
    .merge(df_pools, on="pool_address")
    [['date', 'total_apr', 'pool_symbol']]
) 
d_weekly = (
    d.groupby(['pool_symbol', pd.Grouper(key='date', freq='W-MON')]).mean().reset_index()
)

selection = alt.selection_multi(fields=['pool_symbol'], bind='legend')

(
    alt.Chart(d)
    .mark_line()
    .encode(
        x="date:T", 
        y=alt.Y("total_apr:Q", scale=alt.Scale(domain=(0, 1), clamp=True)), 
        color="pool_symbol:N",
        opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
    )
    .add_selection(selection) 
) | (
    alt.Chart(d_weekly)
    .mark_rect()
    .encode(
        x="date:O", 
        y="pool_symbol:N", 
        color=alt.Color("total_apr:Q", scale=alt.Scale(domain=(0, 1), clamp=True)), 
        tooltip=["date:O", "pool_symbol:N", "total_apr:Q"]
    )
    
)

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [7]:
# TODO: Show top 19 and aggregate others into single "Other" category.
d_mpool_share = (
    df_mpool_snaps[[
        'date', 'pool_address', 'crvfrax_in_mpool', 'crvfrax_in_all_mpools', 'crvfrax_share_mpools', 'crvfrax_share_fraxbp'
    ]]
    .merge(df_pools[['pool_address', 'pool_symbol']], how='left', on='pool_address')
)
d_mpool_share_last = d_mpool_share.loc[d_mpool_share.date == d_mpool_share.date.max()]

### (Chart) Metapool TVL Share 

- Segmented by pool name 

In [8]:
def chart_mpool_tvl_share(): 
    x = alt.X('date:T', title="date")
    color = alt.Color("pool_symbol:N", scale=alt.Scale(range=colors_28))

    chart_share_of_fraxbp = (
        alt.Chart(d_mpool_share)
        .mark_area()
        .encode(
            x=x, 
            y=alt.Y('crvfrax_share_fraxbp:Q', axis=alt.Axis(format=",%", title="% FraxBP")), 
            color=color, 
            tooltip=["pool_symbol:N", alt.Tooltip('crvfrax_share_fraxbp:Q', format=".1%", title='% FraxBP')]
        )
        .properties(title="Historical Metapool % FraxBP")
    ) 
    chart_share_of_fraxbp_current = (
        alt.Chart(d_mpool_share_last)
        .mark_arc()
        .encode(
            theta='crvfrax_share_fraxbp:Q', 
            color=color, 
            tooltip=["pool_symbol:N", alt.Tooltip('crvfrax_share_fraxbp:Q', format=".1%", title='% FraxBP')]
        )
        .properties(title="Current Metapool % FraxBP")
    )

    chart_metapool_share = (
        alt.Chart(d_mpool_share)
        .mark_area()
        .encode(
            x=x, 
            y=alt.Y('crvfrax_share_mpools:Q', axis=alt.Axis(format=",%", title="% Across Metapools"), scale=alt.Scale(domain=[0,1])), 
            color=color, 
            tooltip=["pool_symbol:N", alt.Tooltip('crvfrax_share_mpools:Q', format=".1%", title='% Across Metapools')]
        )
        .properties(title="Historial Metapool Share Across All Metapools")
    )

    return (chart_share_of_fraxbp | chart_share_of_fraxbp_current | chart_metapool_share)

chart_mpool_tvl_share()

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [9]:
mpool_addrs = df_pools.loc[df_pools.pool_fraxbp_metapool == True].pool_address.unique()
df_tvl = (
    # Get share of non crvFRAX in each of the metapools 
    df_reserves.loc[
        df_reserves.pool_address.isin(mpool_addrs) & (df_reserves.pool_coin_address != addresses.token.crvfrax)
    ]
    .merge(df_pools[['pool_address', 'pool_type', 'pool_symbol']], how='left', on='pool_address')
)

x = alt.X("date:T", axis=alt.Axis(title="Timestamp"))
color = alt.Color("pool_symbol:N", scale=alt.Scale(range=colors_28))
facet = alt.Facet('pool_type:N', columns=1, header=alt.Header(title=None, labels=False))

# Charts 
chart_tvl_type_breakdown = (
    alt.Chart(df_tvl)
    .transform_aggregate(groupby=['date', 'pool_type'], tvl_pool_type="sum(reserves_usd)")
    .transform_joinaggregate(groupby=['date'], tvl_total="sum(tvl_pool_type)")
    .encode(
        x=x, 
        tooltip=[
            "date:T", 
            "pool_type:N",
            alt.Tooltip("tvl_pool_type:Q", format="$,d"), 
            alt.Tooltip("tvl_total:Q", format="$,d")
        ]
    )
)
chart_tvl_type_breakdown_area = (
    chart_tvl_type_breakdown
    .mark_area()
    .encode(
        y=alt.Y("tvl_pool_type:Q", axis=alt.Axis(title="TVL ($)")), 
        color="pool_type:N",
    )
)
chart_tvl_type_breakdown_line = (
    chart_tvl_type_breakdown
    .mark_line()
    .encode(y="tvl_total:Q")
)

alt.vconcat(
    alt.hconcat(
        (
            alt.Chart(df_tvl)
            .mark_area()
            .encode(
                x=x,
                y=alt.Y("reserves_usd:Q", axis=alt.Axis(title="TVL ($)")), 
                facet=facet, 
                color=color, 
                tooltip=[
                    "date:T", 
                    "pool_symbol:N", 
                    alt.Tooltip("reserves_usd:Q", format="$,d")
                ]
            )
            .resolve_scale(y="independent").resolve_axis("independent")
            .properties(title="Historical Metapool TVL Breakdown")
        ), 
        (
            alt.Chart(
                df_tvl.loc[df_tvl.date == df_tvl.date.max()]
            )
            .mark_arc()
            .encode(
                theta="reserves_usd:Q", 
                color=color, 
                facet=facet, 
                tooltip=[
                    "pool_symbol:N", 
                    alt.Tooltip("reserves_usd:Q", format="$,d", title="TVL"),
                ] 
            )
            .resolve_scale(theta="independent")
            .properties(title="Current Metapool TVL Breakdown")
        )
    ),
    alt.layer(chart_tvl_type_breakdown_area, chart_tvl_type_breakdown_line), 
    center=True
).resolve_legend(color="independent").resolve_scale(color="independent")

/opt/miniconda3/envs/flywheel/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

### Snapshot Proposals 

We retrieve all snapshot proposals for convex gauge weight snapshots. 

Since votium bribes are intended to get vlCVX holders to vote for particular choices in this snapshot, this data is necessary. 

In [ ]:
from flywheel_util.flows.votium import flow_votium_votes

async with get_client() as client:
    # set a concurrency limit of 10 on the 'small_instance' tag
    limit_id = await client.create_concurrency_limit(tag="network_request", concurrency_limit=1)

(
    df_proposals, 
    df_bribes, 
    df_choices, 
    df_votes, 
    df_epoches, 
    df_prices, 
    df_votium_frax, 
    df_gauge_info, 
    df_claims 
) = await flow_votium_votes()

/Users/brycemorrow/Documents/code/frax-bribes/flywheel_util/src/flywheel_util/tasks/votium.py:424: DeprecationWarning: invalid escape sequence '\)'
  m = re.search('.*\\u2026([^\)]*)\)$', choice)


13:20:02.740 | INFO    | prefect.engine - Created flow run 'greedy-serval' for flow 'flow-votium-votes'

13:20:02.797 | INFO    | Flow run 'greedy-serval' - Created task run 'query_snapshot_proposals-f39e1a07-0' for task 'query_snapshot_proposals'

13:20:02.798 | INFO    | Flow run 'greedy-serval' - Executing 'query_snapshot_proposals-f39e1a07-0' immediately...

13:20:02.808 | INFO    | Flow run 'greedy-serval' - Created task run 'query_bribe_asset_prices-18dc8e38-0' for task 'query_bribe_asset_prices'

...

13:20:35.813 | INFO    | Task run 'blocking_query_snapshot_votes_for_proposal-df415f46-7' - Finished in state Completed()

Requesting page range 0 - 2 / Record Range [0, 2999]
-- Page 1 returned 0 records with page size 1000.
-- Page 0 returned 807 records with page size 1000.
-- Page 2 returned 0 records with page size 1000.


13:20:36.284 | INFO    | Task run 'blocking_query_snapshot_votes_for_proposal-df415f46-31' - Finished in state Completed()

Requesting page range 0 - 2 / Record Range [0, 2999]
-- Page 1 returned 113 records with page size 1000.
-- Page 2 returned 0 records with page size 1000.
-- Page 0 returned 1000 records with page size 1000.


13:20:36.865 | INFO    | Task run 'blocking_query_snapshot_votes_for_proposal-df415f46-21' - Finished in state Completed()

In [ ]:
(
    alt.Chart((
        df_votes[['proposal_round', 'choice', 'choice_vp']]
        .groupby(['proposal_round', 'choice']).sum()
        .reset_index() 
    ))
    .mark_bar()
    .encode(
        x="proposal_round:O", 
        y="choice_vp:Q", 
        color="choice:N", 
        tooltip=["choice:N", "choice_vp:Q"]
    )
    .properties(title="Convex Gauge Weight Vote", width=500) 
)

In [ ]:
color = alt.Color('gauge_short_name:N', scale=alt.Scale(range=colors_28))

def chart_bribes_historical(df): 
    return (
        alt.Chart(df)
        .transform_joinaggregate(groupby=['proposal_round'], bribe_amount_total_usd="sum(bribe_amount_usd)")
        .mark_bar()
        .encode(
            x="proposal_round:O", 
            y="bribe_amount_usd:Q", 
            color=color, 
            tooltip=[
                alt.Tooltip('epoch_end_date:T'), 
                alt.Tooltip('gauge_short_name:N'), 
                alt.Tooltip('bribe_token_name:N'),
                alt.Tooltip('bribe_amount:Q', format=",d"),
                alt.Tooltip('bribe_amount_usd:Q', format="$,d"),
                alt.Tooltip('bribe_amount_total_usd:Q', format="$,d"), 
            ]
        )
    )
    
    
def chart_bribes_last_round(df): 
    return (
        alt.Chart(df)
        .mark_arc()
        .encode(
            theta="bribe_amount_usd:Q", 
            color=color, 
            tooltip=['gauge_short_name:N', alt.Tooltip('bribe_amount_usd:Q', format="$,d")]
        )
    )


data = df_votium_frax.copy()
metapool_gauge_addrs = df_pools.loc[df_pools.pool_fraxbp_metapool == True].dropna(subset='pool_gauge').pool_gauge.unique()
data['is_metapool'] = data.gauge_address.apply(lambda addr: addr in metapool_gauge_addrs)

last_round = data.proposal_round.max()
data_last = data.loc[data.proposal_round == last_round]
data_mp = data.loc[data.is_metapool == True] 
data_mp_last = data_mp.loc[data.proposal_round == last_round]

dfp = df_proposals.copy()
dfp.proposal_end = pd.to_datetime(dfp.proposal_end)

# TODO: Bribe for FraxBP in round 26 was using cvxCRV, why is this? 
(
    (chart_bribes_historical(data) | chart_bribes_last_round(data_last)) & 
    (chart_bribes_historical(data_mp) | chart_bribes_last_round(data_mp_last)).resolve_scale(theta='independent', color="shared")
)

In [ ]:
first_row(df_pool_snaps)
first_row(df_votium_frax)

In [ ]:
first_row(df_epoches) 

In [ ]:
first_row(df_epoches)

In [ ]:
df_ranges = df_epoches[['epoch_start_date']].sort_values('epoch_start_date').reset_index(drop=True)
df_ranges['epoch_start_date_next'] = df_ranges.epoch_start_date.shift(-1).fillna(pd.Timestamp.now())

# Amount of usd bribes per gauge and round (identified by timestamp) 
df_bribe_gauge_round = df_votium_frax.groupby(['gauge_address', 'epoch_start_date', 'epoch_end_date'])['bribe_amount_usd'].sum().reset_index()

# Charts here validated by going to https://curve.fi/#/ethereum/pools
# entering "fraxbp" into the search bar and comparing the tvl of all pools in the UI 
# to the tvl in the charts seen here. 
data = (
    df_mpool_snaps
    .merge(df_pools, how='left', on='pool_address')
    .merge(df_reserves, how='left', on=['pool_address', 'date'])
    .merge(df_coins, how='left', on=['pool_address', 'pool_coin_address'])
    .merge(
        df_bribe_gauge_round, 
        how='left', 
        left_on=['pool_gauge', 'date'], 
        right_on=['gauge_address', 'epoch_end_date'], 
    )
    .sort_values('date').reset_index(drop=True)
)
# For each timestamp, determine the window that it falls within, defined by both a start and end point 
data = pd.merge_asof(
    data, 
    df_ranges.rename(columns={'epoch_start_date': 'window_start'})[['window_start']], 
    left_on='date', 
    right_on='window_start', 
    direction="backward",
    allow_exact_matches=True # Window lower bound should be inclusive 
)
data = pd.merge_asof(
    data, 
    df_ranges.rename(columns={'epoch_start_date_next': 'window_end'})[['window_end']], 
    left_on='date', 
    right_on='window_end', 
    direction="forward", 
    allow_exact_matches=False # Window upper bound should be non-inclusive 
)
data['next_epoch_tvl'] = data.groupby(['pool_name', 'window_start', 'window_end'])['snap_tvl_usd'].transform('mean')
data['tvl_to_bribe_ratio'] = data.next_epoch_tvl / data.bribe_amount_usd

In [ ]:
ncols = 1 
rows = []
row = []
pool_names = data.pool_name.unique().tolist()
groups = list(data.groupby("pool_name"))

def group_sort_key(g): 
    # Sort by tvl at last time point 
    gdf = g[1]
    return -1 * gdf.loc[gdf.date == gdf.date.max(), 'reserves_usd'].sum()

groups.sort(key=group_sort_key)
for pool_name, sdf in groups: 
    sdf = sdf[[
        'pool_name', 'pool_gauge', 'date', 'reserves_usd', 'snap_tvl_usd', 
        'pool_coin_name', 'snap_vol_usd', 'snap_liq_util', 'mpool_paired_asset_vol_usd', 
        'epoch_start_date', 'bribe_amount_usd', 'next_epoch_tvl', 'tvl_to_bribe_ratio'
    ]]
    paired_asset = [e for e in sdf.pool_coin_name.unique() if e != 'crvFRAX'][0]
    dfb = sdf.loc[sdf.pool_coin_name == 'crvFRAX']
    
    # Chart showing TVL in the pool 
    tvl_base = (
        alt.Chart(sdf)
        .transform_calculate(stack_order="datum.pool_coin_name === 'crvFRAX' ? 0 : 1")
        .transform_joinaggregate(groupby=['pool_name', 'date'], tvl_total="sum(reserves_usd)")
        .encode(x="date:T", order="stack_order:O", )
    )
    chart_tvl_area = (
        tvl_base
        .mark_area()
        .encode(
            y=alt.Y("reserves_usd:Q", axis=alt.Axis(title="TVL ($)")), 
            color=alt.Color("pool_coin_name:N", scale=alt.Scale(range=colors_28)), 
            tooltip=[
                alt.Tooltip("pool_coin_name:N"), 
                alt.Tooltip("date:Q", format='$,d'), 
                alt.Tooltip("tvl_total:Q", format='$,d'), 
            ]
        )
    )
    chart_tvl_line = (
        tvl_base
        .transform_filter("datum.pool_coin_name == 'crvFRAX'")
        .mark_line()
        .encode(y="tvl_total:Q")
    )
    chart_pool_liquidity = alt.layer(chart_tvl_area, chart_tvl_line)
    # Chart showing volume in the pool 
    vol_base = (
        alt.Chart(sdf)
        .transform_filter("datum.pool_coin_name !== 'crvFRAX'")
        .encode(x="date:T", order="stack_order:O")
    )
    chart_vol_bar = (
        vol_base
        .mark_bar()
        .encode(
            y="snap_vol_usd:Q", 
            tooltip=[
                alt.Tooltip("snap_vol_usd:Q", format='$,d'), 
            ]
        )
    )
    # Chart showing liquidity utilization in the pool 
    chart_liq_util = (
        vol_base
        .mark_line()
        .encode(
            y=alt.Y("snap_liq_util:Q", scale=alt.Scale(domain=[0,1.0], clamp=True)), 
            tooltip=[
                alt.Tooltip("snap_liq_util:Q", format='$,d'), 
            ]
        )
    )
    # Chart showing ecosystem wide tvl for non crvFRAX assets 
    chart_vol_total = (
        vol_base
        .mark_bar()
        .encode(
            y="mpool_paired_asset_vol_usd:Q", 
            tooltip=[
                alt.Tooltip("mpool_paired_asset_vol_usd:Q", format='$,d'), 
            ]
        )
    )
    # Chart showing ecosystem wide tvl for non crvFRAX assets 
    chart_bribes = (
        vol_base
        .mark_point()
        .encode(
            y="bribe_amount_usd:Q", 
            tooltip=[
                alt.Tooltip("bribe_amount_usd:Q", format='$,d'), 
            ]
        )
    )
    chart_bribe_ratio = (
        alt.Chart(dfb.loc[~dfb.tvl_to_bribe_ratio.isna()])
        .mark_line()
        .encode(
            x="date:T",
            y="tvl_to_bribe_ratio:Q", 
            tooltip=[
                alt.Tooltip("tvl_to_bribe_ratio:Q", format='.2f'), 
            ]
        )
    )

    pool_symbol = pool_name.split(':')[-1].strip()
    w = 250
    h = 150
    
    row.append(
        alt.hconcat(
            chart_pool_liquidity.properties(title=f"TVL: {pool_symbol}", width=w, height=h), 
            chart_liq_util.properties(title=f"Liquidity Utilization (Curve): {pool_symbol}", width=w, height=h), 
            chart_vol_bar.properties(title=f"Volume (Curve): {pool_symbol}", width=w, height=h), 
            chart_vol_total.properties(title=f"Total Volume (Ecosystem): {paired_asset}", width=w, height=h), 
            chart_bribes.properties(title=f"Votium Bribes: {pool_symbol}", width=w, height=h), 
            chart_bribe_ratio.properties(title=f"Votium Bribes Ratio: {pool_symbol}", width=w, height=h), 
            bounds='flush', 
            spacing=75
        )
        .resolve_scale(x="shared")
    )
    if len(row) == ncols: 
        rows.append(row) 
        row = []
    
rows = [
    alt.hconcat(*row) 
    .resolve_scale(x="shared")
    for row in rows
]
chart = (
    alt.vconcat(*rows)
    .resolve_scale(x="shared")
)
chart

In [ ]:
# data.head()

In [ ]:
d = data[['pool_name', 'pool_symbol', 'pool_coin_name', 'window_start', 'window_end', 'next_epoch_tvl', 'bribe_amount_usd', 'tvl_to_bribe_ratio']]
d = d.loc[~d.next_epoch_tvl.isna() & (d.pool_coin_name == 'crvFRAX') & ~d.bribe_amount_usd.isna()].drop_duplicates()
(
    alt.Chart(d)
    .mark_bar()
    .encode(
        x="window_start:O", y="next_epoch_tvl:Q", color=alt.Color("pool_symbol:N", scale=alt.Scale(range=colors_24)), 
        tooltip=["window_start:O", "pool_symbol:N", "next_epoch_tvl:Q"]
    ) 
    | 
    alt.Chart(d)
    .mark_bar()
    .encode(
        x="window_start:O", y="bribe_amount_usd:Q", color=alt.Color("pool_symbol:N", scale=alt.Scale(range=colors_24)), 
        tooltip=["window_start:O", "pool_symbol:N", "bribe_amount_usd:Q"]
    )
    | 
    alt.Chart(d)
    .mark_circle()
    .encode(
        x="window_start:O", 
        y="pool_symbol:N", 
        color=alt.Color("pool_symbol:N", scale=alt.Scale(range=colors_24)), 
        size="tvl_to_bribe_ratio:Q", 
        tooltip=["window_start:O", "pool_symbol:N", "tvl_to_bribe_ratio:Q"]
    )
)